# Advanced Lane Line #
## Distortion Correction ##

In [53]:
import cv2
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
%matplotlib qt
#Read pickle file which included result of camera calibration.
data_file = 'wide_dist_pickle.p'

with open(data_file, mode='rb') as f:
    data = pickle.load(f)
mtx, dist = data['mtx'], data['dist']

image_name = './test_images/test6.jpg'
img = cv2.imread(image_name)
#Function for distortion correction.
def distortion_correction(img, mtx, dist):
    #use above datas, make undistorted image.
    output = cv2.undistort(img, mtx, dist, None, mtx)
    return output

distortion_output = distortion_correction(img, mtx, dist)
#show the image and save it.
cv2.imwrite('./output_images/trans_test6.jpg', distortion_output)

True

## Make Binary And Color Combined Image ##

In [54]:
import numpy as np


img = distortion_output

#Sobel operator function (can use sobelx, sobely)
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if orient == 'x':
        sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F,1,0, ksize = sobel_kernel))
    if orient == 'y':
        sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F,0,1, ksize = sobel_kernel))
    scaled_sobel = np.uint8(255*sobel/np.max(sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

#Function about magnitude of the gradient
def mag_thresh(iamge, sobel_kernel=3, mag_thresh=(0,255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F,1,0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F,0,1, ksize = sobel_kernel)
    mag_sobel = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(mag_sobel)/255
    gradmag = (mag_sobel/scale_factor).astype(np.uint8)
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return mag_binary

#Function about direction of the gradient
def dir_thresh(image, sobel_kernel=3, dir_thresh=(0,np.pi/2)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F,1,0, ksize = sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F,0,1, ksize = sobel_kernel))
    direction = np.arctan2(sobely, sobelx)
    dir_binary = np.zeros_like(direction)
    dir_binary[(direction >= dir_thresh[0]) & (direction <= dir_thresh[1])] = 1
    return dir_binary

#Function for s_channel, getting lane line binary data
def hls_select(image, thresh=(0, 255)):
    #transform the image RGB to HLS.
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

#Function for color and gradient binary images combined thresholds.
def make_binary_image(image, ksize = 13):
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(40,100))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(40,100))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(40,100))
    dir_binary = dir_thresh(image, sobel_kernel= ksize, dir_thresh=(0.2,0.5))
    col_binary = hls_select(image, thresh=(120,240))
                            
    grad_combined = np.zeros_like(dir_binary)
    grad_combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    output = np.zeros_like(grad_combined)
    output[(grad_combined == 1) | (col_binary == 1)] = 1
    return output

all_combined = make_binary_image(img)
plt.imshow(all_combined, cmap = 'gray')
cv2.imwrite('binary_image.jpg', all_combined)

True

## Perspective Transform

In [55]:
image = all_combined
img_size = (image.shape[1], image.shape[0])

#define source area and destination area(perspective transform area).
src = np.float32(
    [[(img_size[0] / 2) - 60, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 60), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])


def warper(img, src, dst):
    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    return warped


warped_img = warper(image, src, dst)

In [56]:
#print out images(one is original added source area line and the other is output added perspective transform area).
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
f.tight_layout()
ax1.imshow(image, cmap = 'gray')
x = [(img_size[0] / 2) - 60,((img_size[0] / 6) - 10),(img_size[0] * 5 / 6) + 60,(img_size[0] / 2 + 60)]
y = [img_size[1] / 2 + 100,img_size[1],img_size[1],img_size[1] / 2 + 100]
ax1.plot(x, y, 'b--', lw=2)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(warped_img, cmap = 'gray')
x = [(img_size[0] / 4),(img_size[0] / 4),(img_size[0] * 3 / 4),(img_size[0] * 3 / 4)]
y = [0,img_size[1],img_size[1],0]
ax2.plot(x, y, 'b--', lw=2)
ax2.set_title('Perspective Transform Image', fontsize=30)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
mpimg.imsave('./warped_curve1.png',warped_img, cmap = 'gray')

## Find Lane Line
#### Use the sliding windows (First)

In [61]:
binary_warped = warped_img

#Function for find line(Frist find or lose previous lane line datas can use this function.)
def find_line_with_windows(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    output = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(output,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(output,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
   
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    output[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    output[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    return output , left_fit, right_fit, left_fitx, right_fitx, ploty

In [62]:
output_img, left_fit, right_fit, left_fitx, right_fitx, ploty = find_line_with_windows(binary_warped)
plt.imshow(output_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

#### Skip the sliding windows
###### If you have previous datas you can use this function

In [7]:
def find_line_without_windows(binary_warped, left_fit, right_fit):
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    # Create an image to draw on and an image to show the selection window
    output = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(output)
    # Color in left and right line pixels
    output[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    output[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (255, 0, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 0, 255))
    output = cv2.addWeighted(output, 1, window_img, 0.3, 0)
    return output, left_fit, right_fit, left_fitx, right_fitx, ploty

In [8]:
result, left_fit, right_fit, left_fitx, right_fitx, ploty = find_line_without_windows(binary_warped, left_fit, right_fit)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='green')
plt.plot(right_fitx, ploty, color='green')
plt.xlim(0, 1280)
plt.ylim(720, 0)


(720, 0)

## Measuring Curvature And Left From Center

In [9]:
#Function for measuring curvation and left from center
def measure_information(ploty, left_fitx, right_fitx):
    y_eval = np.max(ploty)
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    #Calculate distance about vihacle left form center
    bottom_left = 320 - left_fitx[719]
    left_from_center = bottom_left * xm_per_pix
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    #print(left_from_center, 'm')
    return left_from_center, left_curverad, right_curverad

left_from_center, left_curverad, right_curverad = measure_information(ploty, left_fitx, right_fitx)

## Draw Line Areas And Information

In [10]:
#Function for drawing line areas and information text
def draw_info(img, warped_img, left_fitx, right_fitx, ploty, dst, src, left_from_center, average_curverad):
    #Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    Minv = cv2.getPerspectiveTransform(dst, src)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    final_output = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = cv2.addWeighted(final_output, 1, newwarp, 0.3, 0)
    font = cv2.FONT_HERSHEY_SIMPLEX

    cv2.putText(result, 'Radius of Curvature = ' + str(round(average_curverad)) + 'm',(50,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)
    #TODO : need to Change
    cv2.putText(result, 'Vehicle is ' + str(round(left_from_center,2)) + 'm left of center',(50,100), font, 1.5,(255,255,255),2,cv2.LINE_AA)
    return result

result = draw_info(distortion_output, warped_img, left_fitx, right_fitx, ploty, dst, src, left_from_center, (left_curverad+right_curverad)/2)
plt.imshow(result)


## Make Line And Binary Image Saver Class

In [33]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        #radius of curvature of the line in some units
        self.radius_of_curvature = 0

## Test On Video

In [39]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#make object about left and right lane line.
left_line = Line()
right_line = Line()

In [40]:
def process_image(image):
    #datas for distortion correction
    with open(data_file, mode='rb') as f:
        data = pickle.load(f)
    mtx, dist = data['mtx'], data['dist']
    
    #datas for perspective transform
    src = np.float32(
        [[(img_size[0] / 2) - 60, img_size[1] / 2 + 100],
        [((img_size[0] / 6) - 10), img_size[1]],
        [(img_size[0] * 5 / 6) + 60, img_size[1]],
        [(img_size[0] / 2 + 60), img_size[1] / 2 + 100]])
    dst = np.float32(
        [[(img_size[0] / 4), 0],
        [(img_size[0] / 4), img_size[1]],
        [(img_size[0] * 3 / 4), img_size[1]],
        [(img_size[0] * 3 / 4), 0]])
    
    #pipeline for image processing
    dis_img = distortion_correction(image, mtx, dist)
    binary_img = make_binary_image(dis_img)
    warped_img = warper(binary_img, src, dst)
    
    #if Lane line is not detected corretly, use windows.
    if left_line.detected == False or right_line.detected == False:
        output_img, left_fit, right_fit, left_fitx, right_fitx, ploty = find_line_with_windows(warped_img)
        left_from_center, left_curve, right_curve = measure_information(ploty, left_fitx, right_fitx)
        left_line.detected = True       
        right_line.detected = True
        # this part for initializing object member perameters
        if len(left_line.recent_xfitted) == 0 and len(right_line.recent_xfitted) == 0:
            for i in range(6):
                left_line.recent_xfitted.append(left_fitx)
                right_line.recent_xfitted.append(right_fitx)
                left_line.current_fit.append(left_fit)
                right_line.current_fit.append(right_fit)
            left_line.bestx = left_fitx
            right_line.bestx = right_fitx
            left_line.best_fit = left_fit
            right_line.best_fit = right_fit
            left_line.radius_of_curvature = left_curve
            right_line.radius_of_curvature = right_curve
    #if Lane line is dectected corretly, skip using windows.
    else:
        output_img, left_fit, right_fit, left_fitx, right_fitx, ploty = find_line_without_windows(warped_img ,left_line.best_fit, right_line.best_fit)
        left_from_center, left_curve, right_curve = measure_information(ploty, left_line.bestx, right_line.bestx)
    if len(left_line.recent_xfitted) == 7:
        del left_line.current_fit[0]
        del right_line.current_fit[0]
        del left_line.recent_xfitted[0]
        del right_line.recent_xfitted[0]
        
    # if curve is detected incorrectly, just use before frames datas.    
    if left_curve < 400:
        left_line.detected = False
        left_line.recent_xfitted.append(left_line.bestx)
        left_line.current_fit.append(left_line.best_fit)
    # if curve is detected correctly, save the data in object.
    else:
        left_line.recent_xfitted.append(left_fitx)
        left_line.current_fit.append(left_fit)
        left_line.radius_of_curvature = left_curve
        
    temp = np.zeros_like(left_fitx)
    for i in range(len(left_line.recent_xfitted)):
        temp = temp + left_line.recent_xfitted[i]
    left_line.bestx = temp / len(left_line.recent_xfitted)            
    temp = np.zeros_like(left_fit)
    for i in range(len(left_line.current_fit)):
        temp = temp + left_line.current_fit[i]
    left_line.best_fit = temp / len(left_line.current_fit)
        
    # same as left line.
    if right_curve < 400:
        right_line.detected = False
        right_line.recent_xfitted.append(right_line.bestx)
        right_line.current_fit.append(right_line.best_fit)
    else:
        right_line.recent_xfitted.append(right_fitx)
        right_line.current_fit.append(right_fit)
        right_line.radius_of_curvature = right_curve
        
    temp = np.zeros_like(right_fitx)
    for i in range(len(right_line.recent_xfitted)):
        temp = temp + right_line.recent_xfitted[i]
    right_line.bestx = temp / len(right_line.recent_xfitted)
    temp = np.zeros_like(right_fit)
    for i in range(len(right_line.current_fit)):
        temp = temp + right_line.current_fit[i]
    right_line.best_fit = temp / len(right_line.current_fit)
    
    # calculate average curvation between left side and right side.
    aver_curve = (left_line.radius_of_curvature + right_line.radius_of_curvature) / 2     
    result = draw_info(dis_img, warped_img, left_line.bestx, right_line.bestx, ploty, dst, src, left_from_center, aver_curve)
    # This frame is BGR, so transform RGB image.
    result = cv2.cvtColor(result,cv2.COLOR_BGR2RGB)
    return result

In [41]:
final_output = 'p4_test_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
final_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time final_clip.write_videofile(final_output, audio=False)

[MoviePy] >>>> Building video p4_test_video.mp4
[MoviePy] Writing video p4_test_video.mp4





  0%|          | 0/1261 [00:00<?, ?it/s]


  0%|          | 1/1261 [00:00<09:32,  2.20it/s]


  0%|          | 2/1261 [00:00<09:17,  2.26it/s]


  0%|          | 3/1261 [00:01<09:03,  2.32it/s]


  0%|          | 4/1261 [00:01<09:03,  2.31it/s]


  0%|          | 5/1261 [00:02<09:00,  2.32it/s]


  0%|          | 6/1261 [00:02<08:55,  2.34it/s]


  1%|          | 7/1261 [00:03<09:02,  2.31it/s]


  1%|          | 8/1261 [00:03<09:13,  2.26it/s]


  1%|          | 9/1261 [00:03<09:03,  2.30it/s]


  1%|          | 10/1261 [00:04<09:00,  2.31it/s]


  1%|          | 11/1261 [00:04<09:10,  2.27it/s]


  1%|          | 12/1261 [00:05<09:14,  2.25it/s]


  1%|          | 13/1261 [00:05<09:22,  2.22it/s]


  1%|          | 14/1261 [00:06<09:22,  2.22it/s]


  1%|          | 15/1261 [00:06<09:14,  2.25it/s]


  1%|▏         | 16/1261 [00:07<09:27,  2.19it/s]


  1%|▏         | 17/1261 [00:07<09:35,  2.16it/s]


  1%|▏         | 18/1261 [00:07<09:27,  2.19it/s]


  2%|▏         | 19/1261 [0

[MoviePy] Done.
[MoviePy] >>>> Video ready: p4_test_video.mp4 

CPU times: user 16min 25s, sys: 9.6 s, total: 16min 34s
Wall time: 8min 54s
